In [40]:
import casanova
from collections import Counter


In [41]:
reader = casanova.reader("../../data/toflit18_all_flows.csv")

#Filters
year_col = reader.headers["year"]
dirferme_col = reader.headers["customs_region"]
exchange_col = reader.headers["export_import"]
source_col = reader.headers["best_guess_region_prodxpart"]

#Fields
partner_col = reader.headers["partner_grouping"]
value_col = reader.headers["value"]

imports = Counter()
exports = Counter()
for row in reader:
    if row[year_col] == "1789" and row[dirferme_col] == "Marseille" and row[source_col] == "1":
        if row[exchange_col] == "Imports":
            imports[row[partner_col]] += float(row[value_col] or 0)
        elif row[exchange_col] == "Exports":
            exports[row[partner_col]] += float(row[value_col] or 0)

In [42]:
imports

Counter({'Nord': 3283456.5936220004,
         'Hollande': 2046345.67102488,
         'Italie': 43052914.571335405,
         'Amériques': 31740648.0,
         'Levant et Barbarie': 39436933.45469722,
         'Espagne': 8840868.18039737,
         'Portugal': 304088.001869,
         'France': 32135889.123546023,
         "États-Unis d'Amérique": 958093.7048460001,
         "Flandre et autres états de l'Empereur": 823740.383224,
         'Afrique': 10756.0,
         'Angleterre': 1046333.0,
         '[vide]': 2201.0})

In [44]:
exports

Counter({'France': 36427133.12979482,
         'Monde': 24912626.0,
         'Angleterre': 1140719.80401517,
         'Italie': 13691097.147279572,
         'Allemagne': 2052.0,
         'Levant et Barbarie': 13304798.760940531,
         'Espagne': 9863096.925390964,
         'Nord': 1178146.32251836,
         "Flandre et autres états de l'Empereur": 282884.47705189005,
         'Hollande': 895695.7541407199,
         "États-Unis d'Amérique": 239788.98914249995,
         'Amériques': 14542393.0,
         'Afrique': 408552.0,
         '????': 360.0,
         'Asie': 1660684.0})

Counter()